In [1]:
from mofapy2.run.entry_point import entry_point
import pandas as pd
import numpy as np
import mofax as mfx

# initialise the entry point
ent = entry_point()


        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        


In [2]:
# --- Read data ---
proteomics = pd.read_csv("C:/Users/49152/Downloads/Multi-omics/MOFA/input/2_Pseudobulk_different_composition/proteomics_pdbk_different_for_MOFA.csv")
transcriptomics = pd.read_csv("C:/Users/49152/Downloads/Multi-omics/MOFA/input/2_Pseudobulk_different_composition/transcriptomics_pdbk_different_for_MOFA.csv")

# Move "Gene" column into index (like rownames in R)
proteomics_mat = proteomics.set_index("Gene")
transcriptomics_mat = transcriptomics.set_index("Gene")

# --- Reshape to long format ---
proteomics_long = (
    proteomics_mat
    .reset_index()
    .melt(id_vars="Gene", var_name="sample", value_name="value")
    .rename(columns={"Gene": "feature"})
)
proteomics_long["view"] = "Proteomics"
# Make features unique by adding suffix
proteomics_long["feature"] = proteomics_long["feature"].astype(str) + "_prot"

transcriptomics_long = (
    transcriptomics_mat
    .reset_index()
    .melt(id_vars="Gene", var_name="sample", value_name="value")
    .rename(columns={"Gene": "feature"})
)
transcriptomics_long["view"] = "Transcriptomics"
# Make features unique by adding suffix
transcriptomics_long["feature"] = transcriptomics_long["feature"].astype(str) + "_rna"

# --- Combine into one long DataFrame ---
mofa_long = pd.concat([proteomics_long, transcriptomics_long], ignore_index=True)

mofa_long

,feature,sample,value,view
0,Krt1_prot,C10_05J_bulk01,NaN,Proteomics
1,Slc38a2_prot,C10_05J_bulk01,-0.883269,Proteomics
2,Tnc_prot,C10_05J_bulk01,NaN,Proteomics
3,Camk1_prot,C10_05J_bulk01,NaN,Proteomics
4,Krt6a_prot,C10_05J_bulk01,-0.817864,Proteomics
...,...,...,...,...
76995,Med11_rna,SVEC_07J_bulk13,0.750824,Transcriptomics
76996,Riok1_rna,SVEC_07J_bulk13,0.966808,Transcriptomics
76997,Bclaf1_rna,SVEC_07J_bulk13,0.726323,Transcriptomics
76998,Rnaseh2a_rna,SVEC_07J_bulk13,0.596358,Transcriptomics


In [5]:
# Data.frame format
ent.set_data_df(mofa_long, likelihoods = ["gaussian","gaussian"])


No "group" column found in the data frame, we will assume a common group for all samples...


Loaded group='single_group' view='Proteomics' with N=23 samples and D=1000 features...
Loaded group='single_group' view='Transcriptomics' with N=27 samples and D=2000 features...




In [7]:
ent.set_model_options(factors = 8)

Model options:
- Automatic Relevance Determination prior on the factors: False
- Automatic Relevance Determination prior on the weights: True
- Spike-and-slab prior on the factors: False
- Spike-and-slab prior on the weights: True
Likelihoods:
- View 0 (Proteomics): gaussian
- View 1 (Transcriptomics): gaussian




In [9]:
ent.set_train_options(
    convergence_mode = "slow", 
    dropR2 = 0.001, 
    gpu_mode = True, 
    seed = 1
)


GPU mode is activated



In [ ]:
ent.build()

ent.run()

In [ ]:
# Save the output
ent.save(outfile="C:/Users/49152/Downloads/Multi-omics/MOFA/output/pdbk_different_trained_model.hdf5")